<a href="https://colab.research.google.com/github/AlexandreBourrieau/ML/blob/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/Preparation_des_donnees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Création de données avec l'utilisation d'une fenêtre glissante

Commençons par créer un ensemble de 10 valeurs avec l'outils `Dataset` de tensorflow :

In [147]:
import tensorflow as tf
import numpy as np

# Création d'un ensemble de 15 valeurs en 1D
dataset = tf.data.Dataset.range(15)

for valeur in dataset:
  print(valeur.numpy())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


Utilisons maintenant l'outils `window` de la classe `Dataset` afin d'obtenir plusieurs sous-ensembles. Ces sous-ensembles sont créés à l'aide d'une fenêtre glissante appliquée sur les valeurs précédentes. Les paramètres de la fenêtre glissante sont :
 - Taille de la fenêtre : 5
 - Décalage : 1


In [148]:
# Création d'un ensemble de 15 valeurs en 1D
dataset = tf.data.Dataset.range(15)

# Création de sous-ensemble à l'aide d'une fenêtre glissante
dataset = dataset.window(5,shift=1)

i=0
for fenetre in dataset:
  print ("Fenêtre #%d :" %i, end=" ")
  for valeur in fenetre:
    print (valeur.numpy(), end=" ")
  print()
  i = i+1

Fenêtre #0 : 0 1 2 3 4 
Fenêtre #1 : 1 2 3 4 5 
Fenêtre #2 : 2 3 4 5 6 
Fenêtre #3 : 3 4 5 6 7 
Fenêtre #4 : 4 5 6 7 8 
Fenêtre #5 : 5 6 7 8 9 
Fenêtre #6 : 6 7 8 9 10 
Fenêtre #7 : 7 8 9 10 11 
Fenêtre #8 : 8 9 10 11 12 
Fenêtre #9 : 9 10 11 12 13 
Fenêtre #10 : 10 11 12 13 14 
Fenêtre #11 : 11 12 13 14 
Fenêtre #12 : 12 13 14 
Fenêtre #13 : 13 14 
Fenêtre #14 : 14 


Faisons maintenant en sorte que chaque fenêtre possède le même nombre de données (5 données), en éliminant les fenêtres trop petites :

In [150]:
# Création d'un ensemble de 15 valeurs en 1D
dataset = tf.data.Dataset.range(15)

# Création de sous-ensemble à l'aide d'une fenêtre glissante
# Imposons le nombre d'éléments de chaque fenêtre à 5
dataset = dataset.window(5,shift=1,drop_remainder=True)

i=0
for fenetre in dataset:
  print ("Fenêtre #%d :" %i, end=" ")
  for valeur in fenetre:
    print (valeur.numpy(),end=" ")
  print()
  i=i+1

Fenêtre #0 : 0 1 2 3 4 
Fenêtre #1 : 1 2 3 4 5 
Fenêtre #2 : 2 3 4 5 6 
Fenêtre #3 : 3 4 5 6 7 
Fenêtre #4 : 4 5 6 7 8 
Fenêtre #5 : 5 6 7 8 9 
Fenêtre #6 : 6 7 8 9 10 
Fenêtre #7 : 7 8 9 10 11 
Fenêtre #8 : 8 9 10 11 12 
Fenêtre #9 : 9 10 11 12 13 
Fenêtre #10 : 10 11 12 13 14 


# Conversion des fenêtres au format numpy

Commençons par regarder comment obtenir les valeurs contenues dans chaque fenêtre sous forme de vecteur à l'aide de la fonction `batch()` :

In [157]:
# Création d'un ensemble de 15 valeurs en 1D
dataset = tf.data.Dataset.range(15)

# Création de sous-ensemble à l'aide d'une fenêtre glissante
# Imposons le nombre d'éléments de chaque fenêtre à 5
dataset = dataset.window(5,shift=1,drop_remainder=True)

for fenetre in dataset:
    vecteurs = fenetre.batch(5)
    print(list(vecteurs.as_numpy_iterator()))


[array([0, 1, 2, 3, 4])]
[array([1, 2, 3, 4, 5])]
[array([2, 3, 4, 5, 6])]
[array([3, 4, 5, 6, 7])]
[array([4, 5, 6, 7, 8])]
[array([5, 6, 7, 8, 9])]
[array([ 6,  7,  8,  9, 10])]
[array([ 7,  8,  9, 10, 11])]
[array([ 8,  9, 10, 11, 12])]
[array([ 9, 10, 11, 12, 13])]
[array([10, 11, 12, 13, 14])]


Convertissons maintenant chaque vecteur au format numpy avec la fonction `.numpy()` ou bien en utilisant la fonction `asarray()` de numpy

In [156]:
# Création d'un ensemble de 15 valeurs en 1D
dataset = tf.data.Dataset.range(15)

# Création de sous-ensemble à l'aide d'une fenêtre glissante
# Imposons le nombre d'éléments de chaque fenêtre à 5
dataset = dataset.window(5,shift=1,drop_remainder=True)

for fenetre in dataset:
    vecteurs = fenetre.batch(5)
    for vecteur in vecteurs:
      print(np.asarray(vecteur))
#      print(vecteur.numpy())

[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]
[5 6 7 8 9]
[ 6  7  8  9 10]
[ 7  8  9 10 11]
[ 8  9 10 11 12]
[ 9 10 11 12 13]
[10 11 12 13 14]


Un manière plus élégante est d'utiliser les fonctions lambda avec la fonction d'applatissement `flat_map` de tensorflow :

In [161]:
# Création d'un ensemble de 15 valeurs en 1D
dataset = tf.data.Dataset.range(15)

# Création de sous-ensemble à l'aide d'une fenêtre glissante
# Imposons le nombre d'éléments de chaque fenêtre à 5
dataset = dataset.window(5,shift=1,drop_remainder=True)

# Applique la fonction lambda sur le dataset et applatit le résultat
vecteurs = dataset.flat_map(lambda x: x.batch(5))

for vecteur in vecteurs:
  print(vecteur.numpy())


[0 1 2 3 4]
[1 2 3 4 5]
[2 3 4 5 6]
[3 4 5 6 7]
[4 5 6 7 8]
[5 6 7 8 9]
[ 6  7  8  9 10]
[ 7  8  9 10 11]
[ 8  9 10 11 12]
[ 9 10 11 12 13]
[10 11 12 13 14]
